İŞ PROBLEMİ:

Bir tatil sitesi yaptığı satışların bazı özelliklerini kullanarak seviye tabanlı (level based) yeni satış tanımları oluşturmak ve bu yeni satış tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.Örneğin Antalya’dan Herşey Dahil bir otele yoğun bir dönemde gitmek isteyen bir müşterinin ortalama ne kadar kazandırabileceği belirlenmek isteniyor.

Veri Seti Hikayesi: 
Veri seti bir tatil sitesinin yaptığı satışların fiyatlarını ve bu satışlara ait bilgiler içermektedir. Veri seti her satış işleminde oluşan kayıtlardan meydana gelmektedir. Bunun anlamı tablo tekilleştirilmemiştir. Diğer bir ifade ile müşteri birden fazla alışveriş yapmış olabilir.

Değişkenler: 
SaleId : Satış id SaleDate : Satış Tarihi CheckInDate : Müşterinin otele girişi tarihi Price : Satış için ödenen fiyat ConceptName: Otel konsept bilgisi SaleCityName: Otelin bulunduğu şehir bilgisi CInDay:Müşterinin otele giriş günü SaleCheckInDayDiff: Check in ile giriş tarihi gün farkı Season:Otele giriş tarihindeki sezon bilgisi

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 500)

In [3]:
df = pd.read_csv("/Users/ahmetbozkurt/Desktop/github_projeler/kural_tabanlı_sınıflandırma/kural_tabanl_dataset.csv")
df.head()

,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
0,415122,2022-12-03,2022-12-03,79.304029,Herşey Dahil,Antalya,Saturday,0,Low
1,415103,2022-12-03,2022-12-03,45.970696,Yarım Pansiyon,Antalya,Saturday,0,Low
2,404034,2022-09-12,2022-09-13,77.838828,Herşey Dahil,Antalya,Tuesday,1,High
3,415094,2022-12-03,2022-12-10,222.710623,Yarım Pansiyon,İzmir,Saturday,7,Low
4,414951,2022-12-01,2022-12-03,140.476190,Yarım Pansiyon,İzmir,Saturday,2,Low


1.VERİYE GENEL BAKIŞ

In [ ]:
print("##################### Shape #####################")
print(df.shape)
print("##################### Head #####################")
print(df.head())
print("##################### NA #####################")
print(df.isnull().sum())
print("##################### INFO #####################")
print(df.info())

In [ ]:
# Soru 1: Kaç unique şehir vardır?
df["SaleCityName"].nunique()

In [ ]:
# Soru 2: Kaç unique Concept vardır?
df["ConceptName"].unique()

In [ ]:
# Soru 3: Hangi Concept’den kaçar tane satış gerçekleşmiş?
df["ConceptName"].value_counts()

In [ ]:
# Soru 4: Şehirlere göre satışlardan toplam ne kadar kazanılmış?
df.groupby("SaleCityName").agg({"Price": "sum"}).sort_values("Price", ascending=False)

In [ ]:
# Soru 5: Concept türlerine göre göre ne kadar kazanılmış?
df.groupby("ConceptName").agg({"Price": "sum"}).sort_values("Price", ascending=False)

In [ ]:
# Soru 6: Şehirlere göre PRICE ortalamaları nedir?
df.groupby("SaleCityName").agg({"Price": "mean"}).sort_values("Price", ascending=False)

In [ ]:
# Soru 7: Conceptlere göre PRICE ortalamalarınedir?
df.groupby("ConceptName").agg({"Price": "mean"}).sort_values("Price", ascending=False)

In [ ]:
# Soru 8: Şehir-Concept kırılımında PRICE ortalamaları nedir?
df.groupby(["SaleCityName", "ConceptName"]).agg({"Price": "mean"})

2.VERİ MANİPÜLASYONU

In [ ]:
# SaleCheckInDayDiff değişkenini kategorik bir değişkene çeviriniz.
# Aralıklar için "Last Minuters", "Potential Planners", "Planners", "Early Bookers“ isimlerini kullanabilirsiniz.
labels = ["Last Minuters", "Potential Planners", "Planners", "Early Bookers"]
df["Sale_Check_Cat"] = pd.cut(df["SaleCheckInDayDiff"], bins=[0, 7, 30, 90, float("inf")], labels=labels, right=False)
df.head()

In [ ]:
# SaleCityName, ConceptName, Seasons kırılımında ortalama ödenen ücret ve yapılan işlem sayısı cinsinden inceleyiniz ?
df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price": ["mean", "count"]})

In [ ]:
# SaleCityName, ConceptName, Seasons kırılımının çıktısını PRICE'a göre sıralayınız.
# Elde ettiğiniz çıktıyı agg_df olarak kaydediniz.
# Indekste yer alan isimleri değişken ismine çeviriniz.
agg_df = df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price": "mean"}).sort_values("Price", ascending=False)
agg_df = agg_df.reset_index()
agg_df.head()

In [ ]:
# Yeni seviye tabanlı satışları tanımlayınız ve veri setine değişken olarak ekleyiniz.
# Yeni oluşturacağımız Sales_Level_Based değişkeni sırası ile; SaleCityName, ConceptName, Seasons verilerinin her birinin 
# birleşimini gösterecek şekilde yeni değişken oluşturunuz.
agg_df["Sales_Level_Based"] = agg_df[["SaleCityName", "ConceptName", "Seasons"]].apply(lambda x: "_".join(x).upper(), axis=1)
agg_df.head()

In [ ]:
# Müşterileri Price’a göre 4 segmente ayırınız.
# Segmentleri Segment isimlendirmesi ile değişken olarak agg_df’e ekleyiniz.
# Segmentleri betimleyiniz (Segmentlere göre group by yapıp price mean, max, sum’larını alınız).
agg_df["Segment"] = pd.qcut(agg_df["Price"], 4, labels=["D", "C", "B", "A"])
agg_df.groupby(agg_df["Segment"]).agg({"Price": ["mean", "sum", "max"]})

3.POTANSİYEL GETİRİ TAHMİNİ

In [ ]:
# Yeni gelen müşterileri sınıflandırıp, ne kadar gelir getirebileceklerini tahmin ediniz.Örnek müşteri profilleri tanımlayalım.
# Antalya’da herşey dahil ve yüksek sezonda tatil yapmak isteyen bir kişinin ortalama ne kadar gelir kazandırması beklenir?
new_user = "ANTALYA_HERŞEY DAHIL_HIGH"
print(agg_df.loc[agg_df["Sales_Level_Based"] == new_user, "Segment"].values[0])
print(round(agg_df.loc[agg_df["Sales_Level_Based"] == new_user, "Price"].mean(), 3))

# B Segmentindedir.
# 64.92 Price getirisi öngörülür.

In [ ]:
# Girne’de yarım pansiyon bir otele düşük sezonda giden bir tatilci hangi segmentte yer alacaktır?
new_user2 = "GIRNE_YARIM PANSIYON_LOW"
print(agg_df.loc[agg_df["Sales_Level_Based"] == new_user2, "Segment"].values[0])
print(round(agg_df.loc[agg_df["Sales_Level_Based"] == new_user2, "Price"].mean(), 3))

# C Segmentindedir.
# 48.579 Price getirisi öngörülür.

In [ ]:
# Müşteri bilgileri girildiğinde segmentini ve tahmini getirisini hesaplayan bir fonksiyon yazalım.
def estimate(dataframe, column, city, concept, season):
    user = f"{city.upper()}_{concept.upper()}_{season.upper()}"
    print(f"SEGMENT: {dataframe.loc[dataframe[column] == user, 'Segment'].values[0]}")
    print(f"AVERAGE PRICE: {round(dataframe.loc[dataframe[column] == user, 'Price'].values[0], 4)}")

estimate(agg_df, "Sales_Level_Based", "Girne", "Yarım Pansiyon", "Low")